# RFP Response Generation Workflow

<a href="https://colab.research.google.com/github/run-llama/llama_cloud_services/blob/main/examples/parse/report_generation/rfp_response/generate_rfp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows you how to build a workflow to generate a response to an RFP. 

In this scenario, we assume that you are Microsoft, and you are responding to the [JEDI Cloud RFP](https://imlive.s3.amazonaws.com/Federal%20Government/ID151830346965529215587195222610265670631/HQ0034-18-R-0077.pdf) put out by the federal government. The government is using the submitted responses to decide the best vendor for their needs.

![generate_rfp_img](generate_rfp_img.png)

We index a set of relevant documents that Microsoft has - including its annual report, wikipedia page on Microsoft Azure, a slide deck on the government cloud and cybersecurity capabilities. We then help you build an agentic workflow that can ingest an RFP, and generate a response for it in 
a way that adheres to its guidelines.

We use LlamaParse to parse the context documents as well as the RFP document itself.

**NOTE**: If you want to skip the indexing complexity and use LlamaCloud instead, check out the [RFP Example using LlamaCloud](https://github.com/run-llama/llama_cloud_services-demo/blob/main/examples/report_generation/rfp_response/generate_rfp.ipynb).

In [ ]:
import nest_asyncio

nest_asyncio.apply()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Setup

We download the [JEDI RFP template](https://imlive.s3.amazonaws.com/Federal%20Government/ID151830346965529215587195222610265670631/HQ0034-18-R-0077.pdf).

We also download the context documents for Microsoft:
1. Microsoft 2024 10-K 
2. Azure Wikipedia page
3. A slide deck on Microsoft Azure Government
4. Microsoft Digital Defense Report 

In [ ]:
# download JEDI Cloud RFP Template
!wget "https://imlive.s3.amazonaws.com/Federal%20Government/ID151830346965529215587195222610265670631/HQ0034-18-R-0077.pdf" -O data/jedi_cloud_rfp.pdf
# microsoft annual report
!wget "https://www.dropbox.com/scl/fi/4v5dx8dc9yqc8k0yw5g4h/msft_10k_2024.pdf?rlkey=jdyfrsoyb18ztlq5msunmibns&st=9w6bdyvn&dl=1" -O data/msft_10k_2024.pdf
# !wget "https://microsoft.gcs-web.com/static-files/1c864583-06f7-40cc-a94d-d11400c83cc8" -O data/msft_10k_2024.pdf

# azure wikipedia page
!wget "https://www.dropbox.com/scl/fi/7waur8ravmve3fe8nej0k/azure_wiki.pdf?rlkey=icru2w64oylx1p76ftt6y9irv&st=fr87vxob&dl=1" -O data/azure_wiki.pdf
# azure government slide deck
!wget "https://cdn.ymaws.com/flclerks.site-ym.com/resource/resmgr/2017_Fall_Conf/Presentations/2018-10-12_FCCC_Microsoft_Az.pdf" -O data/azure_gov.pdf
# microsoft cybersecurity capabilities
!wget "https://www.dropbox.com/scl/fi/qh00xz29rlom4md8ce675/microsoft_ddr.pdf?rlkey=d868nbnsu1ng41y1chw69y64b&st=24iqemb1&dl=1" -O data/msft_ddr.pdf

We then parse the context documents with LlamaParse - we use multimodal mode in order to extract text from visual data.

In [ ]:
from llama_cloud_services import LlamaParse

# use our multimodal models for extractions
parser = LlamaParse(
    result_type="markdown",
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="anthropic-sonnet-3.5",
)

In [ ]:
from pathlib import Path

data_dir = "data"
data_out_dir = "data_out_rfp"

!mkdir -p {data_dir}
!mkdir -p {data_out_dir}

# note: we skip the rfp doc, which will be indexed as part of the workflow
files = ["azure_gov.pdf", "azure_wiki.pdf", "msft_10k_2024.pdf", "msft_ddr.pdf"]

**Option 1**: If you haven't parsed the files yet, run the block below

In [ ]:
import pickle

file_dicts = {}

for f in files:
    file_base = Path(f).stem
    full_file_path = str(Path(data_dir) / f)

    file_docs = parser.load_data(full_file_path)

    # attach metadata
    for idx, d in enumerate(file_docs):
        d.metadata["file_path"] = f
        d.metadata["page_num"] = idx + 1

    file_dicts[f] = {"file_path": full_file_path, "docs": file_docs}

#### Generate Summaries for each file 

Now that we've loaded the chunks for each file, let's now generate a summary for each file. These summaries will then be attached to the tool descriptions to help inform the agent on which files to query to fetch the right information.

In [ ]:
from llama_index.core import SummaryIndex
from llama_index.llms.openai import OpenAI

summary_llm = OpenAI(model="gpt-4o-mini")

for f in files:
    print(f">> Generate summary for file {f}")
    index = SummaryIndex(file_dicts[f]["docs"])
    response = index.as_query_engine(llm=summary_llm).query(
        "Generate a short 1-2 line summary of this file to help inform an agent on what this file is about."
    )
    print(f">> Generated summary: {str(response)}")
    file_dicts[f]["summary"] = str(response)

In [ ]:
pickle.dump(file_dicts, open(f"{data_out_dir}/tmp_file_dicts.pkl", "wb"))

**Option 2**: If you've already run parsing, the files should be cached. Run the cell below.

In [ ]:
import pickle

file_dicts = pickle.load(open(f"{data_out_dir}/tmp_file_dicts.pkl", "rb"))

### Build Indexes

Once the text nodes are ready, we feed into our vector store, which will index these nodes into Chroma (you're welcome to use our other 40+ vector store integrations if you'd like).

In [ ]:
!pip install llama-index-vector-stores-chroma

In [ ]:
# Run if you want to recreate the index
!rm -rf storage_rfp_chroma

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

persist_dir = "storage_rfp_chroma"

vector_store = ChromaVectorStore.from_params(
    collection_name="rfp_docs", persist_dir=persist_dir
)
index = VectorStoreIndex.from_vector_store(vector_store)

**NOTE**: Don't run the block below if you've already inserted the nodes. Only run if it's your first time!!

In [ ]:
all_nodes = [c for d in file_dicts.values() for c in d["docs"]]
index.insert_nodes(all_nodes)

### Define Retrievers

Define retrievers, one for each file. 

In [ ]:
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)
from llama_index.core.tools import FunctionTool
from typing import List
from llama_index.core.schema import NodeWithScore
from typing import Optional


# function tools
def generate_tool(file: str, file_description: Optional[str] = None):
    """Return a function that retrieves only within a given file."""
    filters = MetadataFilters(
        filters=[
            MetadataFilter(key="file_path", operator=FilterOperator.EQ, value=file),
        ]
    )

    def chunk_retriever_fn(query: str) -> str:
        retriever = index.as_retriever(similarity_top_k=5, filters=filters)
        nodes = retriever.retrieve(query)

        full_text = "\n\n========================\n\n".join(
            [n.get_content(metadata_mode="all") for n in nodes]
        )

        return full_text

    # define name as a function of the file
    fn_name = Path(file).stem + "_retrieve"

    tool_description = f"Retrieves a small set of relevant document chunks from {file}."
    if file_description is not None:
        tool_description += f"\n\nFile Description: {file_description}"

    tool = FunctionTool.from_defaults(
        fn=chunk_retriever_fn, name=fn_name, description=tool_description
    )

    return tool


# generate tools
tools = []
for f in files:
    tools.append(generate_tool(f, file_description=file_dicts[f]["summary"]))

In [ ]:
# validate an existing function
tools[0].metadata

ToolMetadata(description='Retrieves a small set of relevant document chunks from azure_gov.pdf.\n\nFile Description: This file provides an overview of Microsoft Azure Government, highlighting its secure and compliant cloud services tailored for U.S. government entities, along with features related to digital transformation, application innovation, and data-driven intelligence. It also outlines compliance standards, service models, and the commitment to security and privacy.', name='azure_gov_retrieve', fn_schema=<class 'llama_index.core.tools.utils.azure_gov_retrieve'>, return_direct=False)

## Build Workflow

Let's build a workflow that can iterate through the extracted keys/questions from the RFP, and fill them out! 

The user specifies an RFP document as input. The workflow then goes through the following steps:
1. We parse the RFP template using LlamaParse
2. We then extract out the relevant questions we'd want to ask the knowledge base given the instructions in the RFP
3. For each question, we query the knowledge base using a specialized agent to generate a response. The agent is equipped with a set of retrieval tools over the data.
4. We concatenate the questions/answers into a list of dictionaries.
5. Given the question/answer pairs, we feed it along with the source RFP template into a prompt to generate the final report.



In [ ]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Context,
    Workflow,
    step,
)
from llama_index.core.llms import LLM
from typing import Optional
from pydantic import BaseModel
from llama_index.core.schema import Document
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.prompts import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
import logging
import json
import os

_logger = logging.getLogger(__name__)
_logger.setLevel(logging.INFO)


# this is the research agent's system prompt, tasked with answering a specific question
AGENT_SYSTEM_PROMPT = """\
You are a research agent tasked with filling out a specific form key/question with the appropriate value, given a bank of context.
You are given a specific form key/question. Think step-by-step and use the existing set of tools to help answer the question.

You MUST always use at least one tool to answer each question. Only after you've determined that existing tools do not \
answer the question should you try to reason from first principles and prior knowledge to answer the question.

You MUST try to answer the question instead of only saying 'I dont know'.

"""

# This is the prompt tasked with extracting information from an RFP file.
EXTRACT_KEYS_PROMPT = """\
You are provided an entire RFP document, or a large subsection from it. 

We wish to generate a response to the RFP in a way that adheres to the instructions within the RFP, \
including the specific sections that an RFP response should contain, and the content that would need to go \
into each section.

Your task is to extract out a list of "questions", where each question corresponds to a specific section that is required in the RFP response.
Put another way, after we extract out the questions we will go through each question and answer each one \
with our downstream research assistant, and the combined
question:answer pairs will constitute the full RFP response.

You must TRY to extract out questions that can be answered by the provided knowledge base. We provide the list of file metadata below. 

Additional requirements:
- Try to make the questions SPECIFIC given your knowledge of the RFP and the knowledge base. Instead of asking a question like \
"How do we ensure security" ask a question that actually addresses a security requirement in the RFP and can be addressed by the knowledge base.
- Make sure the questions are comprehensive and addresses all the RFP requirements.
- Make sure each question is descriptive - this gives our downstream assistant context to fill out the value for that question 
- Extract out all the questions as a list of strings.


Knowledge Base Files:
{file_metadata}

RFP Full Template:
{rfp_text}

"""

# this is the prompt that generates the final RFP response given the original template text and question-answer pairs.
GENERATE_OUTPUT_PROMPT = """\
You are an expert analyst.
Your task is to generate an RFP response according to the given RFP and question/answer pairs.

You are given the following RFP and qa pairs:

<rfp_document>
{output_template}
</rfp_document>

<question_answer_pairs>
{answers}
</question_answer_pairs>

Not every question has an appropriate answer. This is because the agent tasked with answering the question did not have the right context to answer it.
If this is the case, you MUST come up with an answer that is reasonable. You CANNOT say that you are unsure in any area of the RFP response.


Please generate the output according to the template and the answers, in markdown format.
Directly output the generated markdown content, do not add any additional text, such as "```markdown" or "Here is the output:".
Follow the original format of the template as closely as possible, and fill in the answers into the appropriate sections.
"""


class OutputQuestions(BaseModel):
    """List of keys that make up the sections of the RFP response."""

    questions: List[str]


class OutputTemplateEvent(Event):
    docs: List[Document]


class QuestionsExtractedEvent(Event):
    questions: List[str]


class HandleQuestionEvent(Event):
    question: str


class QuestionAnsweredEvent(Event):
    question: str
    answer: str


class CollectedAnswersEvent(Event):
    combined_answers: str


class LogEvent(Event):
    msg: str
    delta: bool = False


class RFPWorkflow(Workflow):
    """RFP workflow."""

    def __init__(
        self,
        tools,
        parser: LlamaParse,
        llm: LLM | None = None,
        similarity_top_k: int = 20,
        output_dir: str = data_out_dir,
        agent_system_prompt: str = AGENT_SYSTEM_PROMPT,
        generate_output_prompt: str = GENERATE_OUTPUT_PROMPT,
        extract_keys_prompt: str = EXTRACT_KEYS_PROMPT,
        **kwargs,
    ) -> None:
        """Init params."""
        super().__init__(**kwargs)
        self.tools = tools

        self.parser = parser

        self.llm = llm or OpenAI(model="gpt-4o-mini")
        self.similarity_top_k = similarity_top_k

        self.output_dir = output_dir

        self.agent_system_prompt = agent_system_prompt
        self.extract_keys_prompt = extract_keys_prompt

        # if not exists, create
        out_path = Path(self.output_dir) / "workflow_output"
        if not out_path.exists():
            out_path.mkdir(parents=True, exist_ok=True)
            os.chmod(str(out_path), 0o0777)

        self.generate_output_prompt = PromptTemplate(generate_output_prompt)

    @step
    async def parse_output_template(
        self, ctx: Context, ev: StartEvent
    ) -> OutputTemplateEvent:
        # load output template file
        out_template_path = Path(
            f"{self.output_dir}/workflow_output/output_template.jsonl"
        )
        if out_template_path.exists():
            with open(out_template_path, "r") as f:
                docs = [Document.model_validate_json(line) for line in f]
        else:
            docs = await self.parser.aload_data(ev.rfp_template_path)
            # save output template to file
            with open(out_template_path, "w") as f:
                for doc in docs:
                    f.write(doc.model_dump_json())
                    f.write("\n")

        await ctx.set("output_template", docs)
        return OutputTemplateEvent(docs=docs)

    @step
    async def extract_questions(
        self, ctx: Context, ev: OutputTemplateEvent
    ) -> HandleQuestionEvent:
        docs = ev.docs

        # save all_questions to file
        out_keys_path = Path(f"{self.output_dir}/workflow_output/all_keys.txt")
        if out_keys_path.exists():
            with open(out_keys_path, "r") as f:
                output_qs = [q.strip() for q in f.readlines()]
        else:
            # try stuffing all text into the prompt
            all_text = "\n\n".join([d.get_content(metadata_mode="all") for d in docs])
            prompt = PromptTemplate(template=self.extract_keys_prompt)

            file_metadata = "\n\n".join(
                [
                    f"Name:{t.metadata.name}\nDescription:{t.metadata.description}"
                    for t in tools
                ]
            )
            try:
                if self._verbose:
                    ctx.write_event_to_stream(
                        LogEvent(msg=">> Extracting questions from LLM")
                    )

                output_qs = self.llm.structured_predict(
                    OutputQuestions,
                    prompt,
                    file_metadata=file_metadata,
                    rfp_text=all_text,
                ).questions

                if self._verbose:
                    qs_text = "\n".join([f"* {q}" for q in output_qs])
                    ctx.write_event_to_stream(LogEvent(msg=f">> Questions:\n{qs_text}"))

            except Exception as e:
                _logger.error(f"Error extracting questions from page: {all_text}")
                _logger.error(e)

            with open(out_keys_path, "w") as f:
                f.write("\n".join(output_qs))

        await ctx.set("num_to_collect", len(output_qs))

        for question in output_qs:
            ctx.send_event(HandleQuestionEvent(question=question))

        return None

    @step
    async def handle_question(
        self, ctx: Context, ev: HandleQuestionEvent
    ) -> QuestionAnsweredEvent:
        question = ev.question

        # initialize a Function Calling "research" agent where given a task, it can pull responses from relevant tools and synthesize over it
        research_agent = FunctionCallingAgentWorker.from_tools(
            tools, llm=llm, verbose=False, system_prompt=self.agent_system_prompt
        ).as_agent()

        # ensure the agent's memory is cleared
        response = await research_agent.aquery(question)

        if self._verbose:
            # instead of printing the message directly, write the event to stream!
            msg = f">> Asked question: {question}\n>> Got response: {str(response)}"
            ctx.write_event_to_stream(LogEvent(msg=msg))

        return QuestionAnsweredEvent(question=question, answer=str(response))

    @step
    async def combine_answers(
        self, ctx: Context, ev: QuestionAnsweredEvent
    ) -> CollectedAnswersEvent:
        num_to_collect = await ctx.get("num_to_collect")
        results = ctx.collect_events(ev, [QuestionAnsweredEvent] * num_to_collect)
        if results is None:
            return None

        combined_answers = "\n".join([result.model_dump_json() for result in results])
        # save combined_answers to file
        with open(
            f"{self.output_dir}/workflow_output/combined_answers.jsonl", "w"
        ) as f:
            f.write(combined_answers)

        return CollectedAnswersEvent(combined_answers=combined_answers)

    @step
    async def generate_output(
        self, ctx: Context, ev: CollectedAnswersEvent
    ) -> StopEvent:
        output_template = await ctx.get("output_template")
        output_template = "\n".join(
            [doc.get_content("none") for doc in output_template]
        )

        if self._verbose:
            ctx.write_event_to_stream(LogEvent(msg=">> GENERATING FINAL OUTPUT"))

        resp = await self.llm.astream(
            self.generate_output_prompt,
            output_template=output_template,
            answers=ev.combined_answers,
        )

        final_output = ""
        async for r in resp:
            ctx.write_event_to_stream(LogEvent(msg=r, delta=True))
            final_output += r

        # save final_output to file
        with open(f"{self.output_dir}/workflow_output/final_output.md", "w") as f:
            f.write(final_output)

        return StopEvent(result=final_output)

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")
workflow = RFPWorkflow(
    tools,
    parser=parser,
    llm=llm,
    verbose=True,
    timeout=None,  # don't worry about timeout to make sure it completes
)

#### Visualize the workflow

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(RFPWorkflow, filename="rfp_workflow.html")

rfp_workflow.html


## Run the Workflow

Let's run the full workflow and generate the output! 

This will take 5-20 minutes to run and complete. You can inspect the intermediate verbose outputs below as the intermediate questions/answers are generated. The response is streamed back to the user at the end - the response itself is quite long so will take a while to complete! You can also integrate with an observability provider like LlamaTrace/Arize Phoenix in order to view the results.

In [ ]:
handler = workflow.run(rfp_template_path=str(Path(data_dir) / "jedi_cloud_rfp.pdf"))
async for event in handler.stream_events():
    if isinstance(event, LogEvent):
        if event.delta:
            print(event.msg, end="")
        else:
            print(event.msg)

response = await handler
print(str(response))

Running step parse_output_template
Started parsing the file under job_id ad74a9de-c9d2-44b8-ad09-8a7d8baf383f
Step parse_output_template produced event OutputTemplateEvent
Running step extract_questions
Step extract_questions produced no event
Running step handle_question
>> Extracting questions from LLM
>> Questions:
* What are the specific security requirements for the JEDI Cloud as outlined in the RFP, and how can they be addressed using Microsoft Azure Government's compliance standards and security features?
* How does the RFP define the requirements for high availability and failover in cloud services, and what solutions does Microsoft Azure offer to meet these requirements?
* What are the RFP's stipulations regarding data portability and interoperability, and how can Microsoft Azure's services facilitate these processes?
* What are the expectations for program management and oversight as per the RFP, and how can Microsoft Azure's management tools and practices align with these ex